In [1]:
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
from tqdm.notebook import tqdm, trange
# import pulp

In [2]:
pd.set_option('display.max_colwidth', None)

In [ ]:
brakes = pd.read_csv("../data/ideal_brakes.csv")
gearbox = pd.read_csv("../data/ideal_gearbox.csv")
rear_wing = pd.read_csv("../data/ideal_rear_wing.csv")
front_wing = pd.read_csv("../data/ideal_front_wing.csv")
suspension = pd.read_csv("../data/ideal_suspension.csv")
engine = pd.read_csv("../data/ideal_engine.csv")

In [ ]:
parts_list = [brakes, gearbox, rear_wing, front_wing, suspension, engine]
all_parts = pd.concat(parts_list)
trait_list = ["Power", "Aero", "Grip", "Reliability", "Pit Time"]

In [ ]:
index_list = [list(brakes.index), list(gearbox.index), list(rear_wing.index),
              list(front_wing.index), list(suspension.index), list(engine.index)]
index_combinations = product(*index_list)
tunes = []
da_tunes = []

In [ ]:
for combination in tqdm(index_combinations):
    tune_table = []
    part_names = []
    
    for i in range(len(combination)):
        part_numbers = parts_list[i].loc[combination[i], trait_list]
        part_name = parts_list[i].loc[combination[i], "Part"]
        tune_table.append(part_numbers)
        part_names.append(part_name)
    
    tune = pd.DataFrame(tune_table).sum()
    results = {'Parts': part_names,
               'Power': tune[0],
               'Aero': tune[1],
               'Grip': tune[2],
               'Reliability': tune[3],
               'Pit_Time': tune[4]}
    
    tunes.append(results)
    
tunes = pd.DataFrame(tunes)
tunes.to_csv("../data/ideal_tunes.csv", index=False)

# Use notebook from here on if looking for tunes

In [6]:
tunes = pd.read_csv("../data/ideal_tunes.csv")

### Specific tunes

In [10]:
tunes['overall'] = tunes['Power']+tunes['Aero']+tunes['Grip']+tunes['Reliability']
tunes['setup_MM_value'] = 3*(tunes['Power']+tunes['Aero']+tunes['Grip']+tunes['Reliability']) + 90*(6-tunes['Pit_Time'])

In [11]:
f = tunes['setup_MM_value'] > np.percentile(tunes['setup_MM_value'], 99.99)
# f = tunes['weighted_avg'] == max(tunes['weighted_avg'])
tunes.loc[f].sort_values(by='overall', ascending=False)

,Parts,Power,Aero,Grip,Reliability,Pit_Time,overall,setup_MM_value
2408,"['Clamp', 'The Buffer', 'Hurricane', 'Magnet', 'The Hold', 'Powerplant']",75.0,74.0,99.0,63.0,3.43,311.0,1164.3
2401,"['Clamp', 'The Buffer', 'Hurricane', 'Magnet', 'Swish', 'Powerplant']",74.0,84.0,88.0,62.0,3.27,308.0,1169.7
2744,"['Clamp', 'The Buffer', 'C.X.2', 'Magnet', 'Swish', 'Powerplant']",79.0,74.0,90.0,65.0,3.31,308.0,1166.1
36022,"['Slow Motion', 'The Buffer', 'Hurricane', 'Magnet', 'The Hold', 'Powerplant']",73.0,76.0,94.0,65.0,3.36,308.0,1161.6
52829,"['The Anvil', 'The Buffer', 'Hurricane', 'Magnet', 'The Hold', 'Powerplant']",72.0,76.0,79.0,81.0,3.35,308.0,1162.5
2402,"['Clamp', 'The Buffer', 'Hurricane', 'Magnet', 'Swish', 'Destroyer']",65.0,92.0,88.0,62.0,3.31,307.0,1163.1
36015,"['Slow Motion', 'The Buffer', 'Hurricane', 'Magnet', 'Swish', 'Powerplant']",72.0,86.0,83.0,64.0,3.20,305.0,1167.0
36358,"['Slow Motion', 'The Buffer', 'C.X.2', 'Magnet', 'Swish', 'Powerplant']",77.0,76.0,85.0,67.0,3.24,305.0,1163.4
52822,"['The Anvil', 'The Buffer', 'Hurricane', 'Magnet', 'Swish', 'Powerplant']",71.0,86.0,68.0,80.0,3.19,305.0,1167.9
53165,"['The Anvil', 'The Buffer', 'C.X.2', 'Magnet', 'Swish', 'Powerplant']",76.0,76.0,70.0,83.0,3.23,305.0,1164.3


In [7]:
list(tunes.loc[f, 'Parts'])

["['Clamp', 'The Buffer', 'Hurricane', 'Magnet', 'The Hold', 'Powerplant']",
 "['Clamp', 'The Buffer', 'Hurricane', 'Magnet', 'The Hold', 'Destroyer']",
 "['Clamp', 'The Buffer', 'Hurricane', 'Magnet', 'Joist', 'Powerplant']",
 "['Clamp', 'The Buffer', 'Hurricane', 'Padlock', 'The Hold', 'Powerplant']",
 "['Clamp', 'The Buffer', 'C.X.2', 'Magnet', 'The Hold', 'Powerplant']",
 "['Clamp', 'The Buffer', 'C.X.2', 'Magnet', 'The Hold', 'Destroyer']",
 "['Clamp', 'The Buffer', 'C.X.2', 'Magnet', 'Joist', 'Powerplant']",
 "['Clamp', 'The Buffer', 'C.X.2', 'Padlock', 'The Hold', 'Powerplant']"]

# Drivers

In [8]:
drivers = pd.read_csv("../data/ideal_drivers.csv")
drivers['dry_qual'] = drivers['Overtaking']+drivers['Defending']+drivers['Consistency']+drivers['Fuel']+drivers['Tyre']
drivers['wet_qual'] = drivers['Overtaking']+drivers['Defending']+drivers['Consistency']+drivers['Fuel']+drivers['Tyre']+drivers['Rain']
drivers['conservators'] = drivers['Fuel']+drivers['Tyre']

In [9]:
drivers.sort_values(by='wet_qual', ascending=False)

,Drivers,Overtaking,Defending,Consistency,Fuel,Tyre,Rain,dry_qual,wet_qual,conservators
0,Hamilton,113,101,107,95,88,120,504,624,183
1,Verstappen,120,101,88,113,95,107,517,624,208
2,Ricciardo,112,87,118,93,100,106,510,616,193
3,Leclerc,112,100,87,106,118,93,523,616,224
4,Bottas,98,105,111,117,92,86,523,609,209
5,Perez,86,92,111,105,98,117,492,609,203
6,Sainz,98,111,117,86,105,92,517,609,191
7,Russell,86,92,105,98,117,111,498,609,215
8,Gasly,116,85,97,103,91,110,492,602,194
9,Alonso,103,116,110,85,97,91,511,602,182
